In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

# CIFAR-10 학습데이터 준비

In [ ]:
# CIFAR-10 데이터셋 준비
import tensorflow.keras.datasets as ds

(x_train,y_train),(x_test,y_test) = ds.cifar10.load_data()

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
# 학습 데이터 정규화
# CIFAR-10 훈련 샘플 50000개, 테스트 샘플 10000개
# 이미지 크기: 32 * 32 * 3 = 3072

# 1차원 구조로 변경 (Flattening)
# x_train.reshape(샘플 수, 3072) 또는 (샘플 수, -1)
x_trainF = x_train.reshape(x_train.shape[0], 3072)
x_testF = x_test.reshape(x_test.shape[0], 3072)

# 0~1로 정규화하기 위해 실수형으로 변환
x_trainF = x_trainF.astype('float32')
x_testF = x_testF.astype('float32')

# 0~1로 정규화
x_trainF /= 255.0
x_testF /= 255.0

# 수정된 Shape 출력
print(x_trainF.shape, x_testF.shape)

In [ ]:
y_train10 = tf.keras.utils.to_categorical(y_train,10)   # one-hot 인코딩
y_test10 = tf.keras.utils.to_categorical(y_test,10)

y_train10.shape, y_test10.shape

# 인공신경망(Artificial Neural Network, ANN)

### 신경망 학습 : 예, MLP

In [ ]:
# 학습 모델 : 신경망
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

# Input layer의 shape 3072
mlp_model = Sequential()
mlp_model = Sequential([
    Input(shape=(3072,)),
    Dense(512, activation='relu'),  # 층 너비 증가
    Dense(256, activation='relu'),  # 은닉층 추가
    Dense(10, activation='softmax')
])

mlp_model.summary()

In [ ]:
# 학습
from tensorflow.keras.optimizers import SGD

# 모델 컴파일
# 손실 함수 'categorical_crossentropy' 사용
# 학습률 0.01 >> 0.04
mlp_model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(learning_rate=0.04),
                  metrics=['accuracy'])

# 학습
history = mlp_model.fit(x_trainF,
                        y_train10,
                        batch_size=128,
                        epochs=50,
                        validation_data=(x_testF, y_test10),
                        verbose=2)
# 첫 시도: 정확도 0.48
# 2차 시도: 정확도 0.52...테스트 정확도 0.51

In [ ]:
# plot loss and acc
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('Loss value')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epoch')
plt.ylabel('acc value')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# 평가 : 학습이 끝난 후 테스트 데이터로 모델의 정확도 측정
mlp_acc = mlp_model.evaluate(x_testF, y_test10, verbose=0)
print('정확도 = ', mlp_acc[1]*100)

In [ ]:
# 예측 : 학습 모델에 입력데이터에 대한 출력 예측

input_data = x_testF[0].reshape(1, 3072)

output = mlp_model.predict(input_data)
print(output)

# 가장 큰 확률의 클래스(라벨)
top1 = output.argmax(axis=1)
print(top1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    # 1. 이미지 표시
    plt.imshow(x_test[i])

    # 2. 예측 수행
    predicted_label = mlp_model.predict(x_testF[i].reshape(1, 3072), verbose=0).argmax(axis=1)[0]
    true_label = y_test[i][0] if y_test[i].ndim > 1 else y_test[i]

    # 라벨 표시
    plt.xlabel(f"정답: {true_label} \n 예측: {predicted_label}")

plt.show()

# 딥러닝 : CNN

## LeNet5

### CNN 학습을 위한 학습데이터

In [ ]:
# 2차원 구조(vs. mlp)
x_trainC = x_train.astype(np.float32)/255.0  # 0~1로 정규화
x_testC = x_test.astype(np.float32)/255.0

x_trainC.shape, x_testC.shape

In [ ]:
y_train10 = tf.keras.utils.to_categorical(y_train,10)   # one-hot 인코딩
y_test10 = tf.keras.utils.to_categorical(y_test,10)

y_train10.shape, y_test10.shape

### CNN 학습

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

cnn = Sequential()
# 1. Input Shape 수정: CIFAR-10에 맞춰 (32, 32, 3)으로 변경
cnn.add(Conv2D(6, (5, 5), padding='same',use_bias=False, input_shape=(32, 32, 3)))
cnn.add(BatchNormalization()) # 배치 정규화 추가
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Conv2D Layer 2
cnn.add(Conv2D(16, (5, 5), padding='valid',use_bias=False))
cnn.add(BatchNormalization()) # 배치 정규화 추가
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2))

cnn.add(Flatten())

# 3차 시도에서 Droupout 추가(과적합 방지)
# 4차 시도에서 배치 정규화 추가
# 5차 시도에서 dropout 제거

# Dense Layer 1
cnn.add(Dense(units=120, activation='relu'))
#cnn.add(Dropout(0.5)) # 50%의 뉴런을 무작위로 비활성화
# Dense Layer 2
cnn.add(Dense(units=84, activation='relu'))
#cnn.add(Dropout(0.5)) # 50%의 뉴런을 무작위로 비활성화
# Output Layer
cnn.add(Dense(units=10, activation='softmax'))


In [ ]:
cnn.summary()

In [ ]:
#학습
# 모델 컴파일
cnn.compile(loss='categorical_crossentropy',
            optimizer=Adam(learning_rate=0.003),
            metrics=['accuracy'])

# 학습 시작
history = cnn.fit(x_trainC,
                  y_train10,
                  batch_size=64,
                  epochs=50,
                  validation_data=(x_testC, y_test10),
                  verbose=2)

#2차 시도 배치 사이즈 128 >> 64, 학습율 0.01 >> 0.03
#3차 시도 배치 사이즈 복구
#5차 시도 배치 사이즈 64

In [ ]:
# 평가
# 1차 시도: 정확도 60
# 2차 시도: 배치 사이즈, 학습율 조정 후 정확도 58
# 3차 시도: 모델에 dropout추가, 배치 사이즈 다시 128로 복구 후 정확도 62
# 4차 시도: 배치 정규화 추가 후 정확도 60
# 5차 시도: dropout 삭제, 배치 사이즈 64로 재조정 후 정확도 62
cnn_acc = cnn.evaluate(x_testC,y_test10, verbose=0)
print('정확률=',cnn_acc[1]*100)

In [ ]:
# 예측
output = cnn.predict(x_testC[0].reshape(1, 32,32,3))

top1 = output.argmax(axis=1)   # 가장 큰 확률의 클래스(라벨)
print(output)
print(top1)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_test[i], cmap=plt.cm.binary)
    plt.xlabel(str(y_test[i]) + '-->' + str(cnn.predict(x_testC[i].reshape(1,32,32,3)).argmax(axis=1)))
plt.show()